## Gate cutting to reduce circuit width

In [1]:
import numpy as np
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import PauliList

Specify a circuit and some observables

In [2]:
circuit = EfficientSU2(4, entanglement="linear", reps=2).decompose()
circuit.assign_parameters([0.4] * len(circuit.parameters), inplace=True)
observables = PauliList(["ZZII", "IZZI", "IIZZ", "XIXI", "ZIZZ", "IXIX"])

Cut the circuit and observables by using `partition_problem` and specifying qubit partitions

In [3]:
from circuit_knitting.cutting import partition_problem

partitioned_problem = partition_problem(
    circuit=circuit,
    partition_labels="AABB",
    num_samples=np.inf,
    observables=observables,
)
partitioned_problem._fields

('subcircuits', 'subexperiments', 'subobservables', 'weights', 'bases')

Run the subexperiments for each partition using the Qiskit Aer primitives.

In [4]:
from qiskit_aer.primitives import Sampler

# Set up Qiskit Aer Sampler primitives.
sampler_a = Sampler(run_options={"shots": 2**12})
sampler_b = Sampler(run_options={"shots": 2**12})

# Retrieve the quasi-probability distributions for each subexperiment
results_a = (
    sampler_a.run(partitioned_problem.subexperiments["A"]).result()
)
results_b = (
    sampler_b.run(partitioned_problem.subexperiments["B"]).result()
)

Prepare the results objects for reconstruction.

Include the number of clbits used for QPD measurements in each partition as metadata. This will be needed during reconstruction.

In [5]:
for i in range(len(results_a.metadata)):
    results_a.metadata[i]['num_qpd_bits'] = len(partitioned_problem.subexperiments["A"][i].cregs[0])
for i in range(len(results_b.metadata)):
    results_b.metadata[i]['num_qpd_bits'] = len(partitioned_problem.subexperiments["B"][i].cregs[0])

results = {"A": results_a, "B": results_b}

To run using Qiskit Runtime primitives, rather than Qiskit Aer, replace the code above with this commented block.

In [6]:
#from qiskit_ibm_runtime import Session, Options, Sampler

#with Session(backend="ibmq_qasm_simulator") as session:
#    options = Options()
#    options.execution.shots = 2**14
#    sampler_a = Sampler(options=options)
#    sampler_b = Sampler(options=options)

#    job_a = sampler_a.run(partitioned_problem.subexperiments["A"])
#    job_b = sampler_b.run(partitioned_problem.subexperiments["B"])

#    quasi_dists_a = job_a.result()
#    quasi_dists_b = job_b.result()

#    session.close()

Reconstruct the full expectation value, given the results of cutting

In [7]:
from circuit_knitting.cutting import reconstruct_expectation_values

simulated_expvals = reconstruct_expectation_values(
    partitioned_problem.subobservables,
    partitioned_problem.weights,
    results,
)
simulated_expvals

[0.3804730176925658,
 0.5288630723953246,
 0.5727139711380003,
 0.11992454528808594,
 0.3004816770553588,
 -0.1273291707038879]

In [8]:
import qiskit.tools.jupyter  # noqa: F401

%qiskit_version_table